In [11]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [12]:
# URL of page to be scraped
url = 'https://www.surfline.com/surf-reports-forecasts-cams/costa-rica/3624060'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [13]:
# Examine the results, then determine element that contains sought info
results = soup.find_all('div', class_="sl-spot-list__ref")

In [14]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [15]:
# Define database and collection
db = client.costa_rica_db
collection = db.surf_summary

In [17]:
# extract some data from first page
for result in results:
    try:
        location_name = result.find('h3', class_='sl-spot-details__name').text
        wave_height = result.find('span', class_='quiver-surf-height').text
        link = result.a['href']
        
        # extract the water abd air tempreture from the second page
        response = requests.get('https://www.surfline.com'+link)
        soup2 = BeautifulSoup(response.text, 'html.parser')
        results2 = soup2.find('div', class_ = 'sl-wetsuit-recommender__weather')
        water_temp = results2.div.text
        air_temp = results2.find_all("div")[1].text
    
        if (location_name and wave_height and link and water_temp and air_temp):
            print('-------------')
            print('City: '+ location_name)
            print('Water Tempreture: '+ water_temp)
            print('Air Tempreture: '+ air_temp)
            print('Waves Height: '+ wave_height)
            print('https://www.surfline.com'+ link)
            
            
            # Dictionary to be inserted as a MongoDB document
            post = {  'location': location_name,
                      'water_temp': water_temp,
                      'air_temp' : air_temp,
                      'wave_height': wave_height,
                      'url': 'https://www.surfline.com'+link   }

            collection.insert_one(post)
            
    except AttributeError as e:
        print(e)

-------------
City: Camaronal
Water Tempreture: 86 - 88 ºF
Air Tempreture: 80 ºF
Waves Height: 6-8FT
https://www.surfline.com/surf-report/camaronal/584204204e65fad6a7709140
-------------
City: Tamarindo
Water Tempreture: 86 - 88 ºF
Air Tempreture: 82 ºF
Waves Height: 4-6FT
https://www.surfline.com/surf-report/tamarindo/5842041f4e65fad6a7708b9b
'NoneType' object has no attribute 'text'
-------------
City: Witches Rock
Water Tempreture: 86 - 88 ºF
Air Tempreture: 82 ºF
Waves Height: 3-5FT
https://www.surfline.com/surf-report/witches-rock/5842041f4e65fad6a7708b99
-------------
City: Samara
Water Tempreture: 86 - 88 ºF
Air Tempreture: 80 ºF
Waves Height: 4-5FT
https://www.surfline.com/surf-report/samara/584204204e65fad6a770913d
-------------
City: Santa Teresa
Water Tempreture: 86 - 88 ºF
Air Tempreture: 81 ºF
Waves Height: 3-5FT
https://www.surfline.com/surf-report/santa-teresa/5842041f4e65fad6a7708e31
-------------
City: Playa Esterillos
Water Tempreture: 86 - 88 ºF
Air Tempreture: 80 ºF